In [260]:
import pandas as pd
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [250]:
df = pd.read_csv('lego_Sets.csv')

In [251]:
df.list_price = df.list_price.astype(float)
df['review_difficulty'] = df['review_difficulty'].astype("category")
df['review_difficulty'] = df['review_difficulty'].cat.reorder_categories(['Very Easy',
                                                               'Easy',
                                                               'Average',
                                                               'Challenging',
                                                               'Very Challenging'],
                                                                        ordered = True)
df['review_difficulty'] = df['review_difficulty'].cat.codes
df.theme_name = df.theme_name.astype("category")
df.ages = df.ages.astype("category")


In [252]:
df1 = df.copy().dropna()
df1 = df1.drop(['prod_desc', 
                'prod_id', 
                'prod_long_desc', 
                'set_name',
                'theme_name',
                'country'], 
                 axis=1)
df2 = pd.get_dummies(df1)

In [253]:
X, Y = df2.drop(['list_price'], axis=1), df2['list_price']

In [254]:
X_train, X_test, Y_train, Y_test = train_test_split(
                                                    X, 
                                                    Y, 
                                                    test_size=0.3, 
                                                    random_state=361)

In [255]:
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)



# with statsmodels
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)# adding a constant
 
model = sm.OLS(Y_train, X_train).fit()
Y_pred = model.predict(X_test) 
 
print_model = model.summary()
print(print_model)

Intercept: 
 30.901344531178218
Coefficients: 
 [ 1.99713057e-01  1.01323505e-01  1.12535631e+01  5.99413873e+00
 -2.64504311e+00 -1.66857111e+01  2.06111404e+01  1.28232295e+01
  4.73645085e+00  3.37507799e-14 -1.32384418e+01 -4.39353767e+00
 -6.13849638e+01  1.16130596e+01 -7.95315638e+01  1.84696563e+01
  1.12063828e+01  4.14989830e+01 -1.95749861e+01  1.23728216e+01
 -4.28254664e+00  4.49863005e-01  1.74470886e+00  2.71896236e+01
 -6.36881915e-01  4.80532815e+00 -1.19826474e+00  6.95260735e+00
  4.83318992e+00  4.14707702e+00  8.75302544e+00  4.68442572e+00
  4.96721051e+00 -1.61465773e+01 -1.11526757e+01 -8.10713244e-01
  1.04923692e+01]
                            OLS Regression Results                            
Dep. Variable:             list_price   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     1257.
Date:                Tue, 04 Jun 

In [ ]:
df_forecast = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
df_forecast

In [257]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

Mean Absolute Error: 16.700803920243782
Mean Squared Error: 926.5778843273896
Root Mean Squared Error: 30.439741857108277


In [ ]:
X.hist(figsize = (12,10))
plt.show()



In [ ]:
plt.scatter(df1['review_difficulty'], df1['list_price'], color='green')
plt.title('List price vs Review difficulty', fontsize=14)
plt.xlabel('Review difficulty', fontsize=14)
plt.ylabel('List price', fontsize=14)
plt.grid(True)
plt.show()

plt.scatter(df1['num_reviews'], df1['list_price'], color='green')
plt.title('List price vs Number of reviews', fontsize=14)
plt.xlabel('Number of reviews', fontsize=14)
plt.ylabel('List price', fontsize=14)
plt.grid(True)
plt.show()

plt.scatter(df1['piece_count'], df1['list_price'], color='green')
plt.title('List price vs Piece count', fontsize=14)
plt.xlabel('Piece count', fontsize=14)
plt.ylabel('List price', fontsize=14)
plt.grid(True)
plt.show()

plt.scatter(df1['play_star_rating'], df1['list_price'], color='green')
plt.title('List price vs Play star rating', fontsize=14)
plt.xlabel('Play star rating', fontsize=14)
plt.ylabel('List price', fontsize=14)
plt.grid(True)
plt.show()

In [261]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2,  10**-1])}})
# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 
                                                                        'mean_squared_error'])
NN_model.summary()

NameError: name 'choice' is not defined

In [228]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [229]:
NN_model.fit(X_train, Y_train, epochs=1000, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


Train on 5706 samples, validate on 1427 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 180us/step - loss: 1834.6157 - mean_absolute_error: 21.3786 - mean_squared_error: 1834.6157 - val_loss: 2267.8549 - val_mean_absolute_error: 22.3451 - val_mean_squared_error: 2267.8549

Epoch 00001: val_loss improved from inf to 2267.85486, saving model to Weights-001--2267.85486.hdf5
Epoch 2/1000
5706/5706 [==============================] - 1s 113us/step - loss: 1487.7217 - mean_absolute_error: 19.2109 - mean_squared_error: 1487.7217 - val_loss: 1508.0178 - val_mean_absolute_error: 18.7797 - val_mean_squared_error: 1508.0178

Epoch 00002: val_loss improved from 2267.85486 to 1508.01782, saving model to Weights-002--1508.01782.hdf5
Epoch 3/1000
5706/5706 [==============================] - 1s 114us/step - loss: 1552.1116 - mean_absolute_error: 19.3523 - mean_squared_error: 1552.1116 - val_loss: 1934.8112 - val_mean_absolute_error: 19.8656 - val_mean_squared_error: 1934.8112

Epoc


Epoch 00026: val_loss did not improve from 1290.08664
Epoch 27/1000
5706/5706 [==============================] - 1s 132us/step - loss: 1155.1116 - mean_absolute_error: 15.7924 - mean_squared_error: 1155.1116 - val_loss: 1435.2810 - val_mean_absolute_error: 15.7174 - val_mean_squared_error: 1435.2810

Epoch 00027: val_loss did not improve from 1290.08664
Epoch 28/1000
5706/5706 [==============================] - 1s 147us/step - loss: 1158.4675 - mean_absolute_error: 16.0055 - mean_squared_error: 1158.4675 - val_loss: 1182.6539 - val_mean_absolute_error: 15.2652 - val_mean_squared_error: 1182.6539

Epoch 00028: val_loss improved from 1290.08664 to 1182.65394, saving model to Weights-028--1182.65394.hdf5
Epoch 29/1000
5706/5706 [==============================] - 1s 134us/step - loss: 1208.1761 - mean_absolute_error: 16.2909 - mean_squared_error: 1208.1761 - val_loss: 2705.7721 - val_mean_absolute_error: 23.5870 - val_mean_squared_error: 2705.7721

Epoch 00029: val_loss did not improve fr

5706/5706 [==============================] - 1s 109us/step - loss: 746.0706 - mean_absolute_error: 13.3690 - mean_squared_error: 746.0706 - val_loss: 847.6534 - val_mean_absolute_error: 13.7891 - val_mean_squared_error: 847.6534

Epoch 00053: val_loss improved from 881.84027 to 847.65345, saving model to Weights-053--847.65345.hdf5
Epoch 54/1000
5706/5706 [==============================] - 1s 109us/step - loss: 695.8072 - mean_absolute_error: 13.0744 - mean_squared_error: 695.8072 - val_loss: 1184.4073 - val_mean_absolute_error: 14.0754 - val_mean_squared_error: 1184.4073

Epoch 00054: val_loss did not improve from 847.65345
Epoch 55/1000
5706/5706 [==============================] - 1s 107us/step - loss: 805.8696 - mean_absolute_error: 13.5219 - mean_squared_error: 805.8696 - val_loss: 1103.6154 - val_mean_absolute_error: 14.1532 - val_mean_squared_error: 1103.6154

Epoch 00055: val_loss did not improve from 847.65345
Epoch 56/1000
5706/5706 [==============================] - 1s 109us/


Epoch 00079: val_loss did not improve from 683.40669
Epoch 80/1000
5706/5706 [==============================] - 1s 108us/step - loss: 655.6160 - mean_absolute_error: 12.2635 - mean_squared_error: 655.6160 - val_loss: 775.6016 - val_mean_absolute_error: 13.1520 - val_mean_squared_error: 775.6016

Epoch 00080: val_loss did not improve from 683.40669
Epoch 81/1000
5706/5706 [==============================] - 1s 109us/step - loss: 571.7257 - mean_absolute_error: 12.1938 - mean_squared_error: 571.7257 - val_loss: 735.6296 - val_mean_absolute_error: 13.4677 - val_mean_squared_error: 735.6296

Epoch 00081: val_loss did not improve from 683.40669
Epoch 82/1000
5706/5706 [==============================] - 1s 106us/step - loss: 545.4746 - mean_absolute_error: 11.9426 - mean_squared_error: 545.4746 - val_loss: 615.4468 - val_mean_absolute_error: 12.8335 - val_mean_squared_error: 615.4468

Epoch 00082: val_loss improved from 683.40669 to 615.44683, saving model to Weights-082--615.44683.hdf5
Epoc

5706/5706 [==============================] - 1s 110us/step - loss: 570.2046 - mean_absolute_error: 11.8060 - mean_squared_error: 570.2046 - val_loss: 732.3482 - val_mean_absolute_error: 11.8715 - val_mean_squared_error: 732.3482

Epoch 00106: val_loss did not improve from 539.56739
Epoch 107/1000
5706/5706 [==============================] - 1s 105us/step - loss: 493.1374 - mean_absolute_error: 10.9422 - mean_squared_error: 493.1374 - val_loss: 769.9488 - val_mean_absolute_error: 11.8021 - val_mean_squared_error: 769.9488

Epoch 00107: val_loss did not improve from 539.56739
Epoch 108/1000
5706/5706 [==============================] - 1s 107us/step - loss: 475.8886 - mean_absolute_error: 10.7965 - mean_squared_error: 475.8886 - val_loss: 639.8083 - val_mean_absolute_error: 11.7282 - val_mean_squared_error: 639.8083

Epoch 00108: val_loss did not improve from 539.56739
Epoch 109/1000
5706/5706 [==============================] - 1s 106us/step - loss: 500.1293 - mean_absolute_error: 11.3939

...

5706/5706 [==============================] - 1s 108us/step - loss: 138.2486 - mean_absolute_error: 6.2960 - mean_squared_error: 138.2486 - val_loss: 306.4869 - val_mean_absolute_error: 7.7933 - val_mean_squared_error: 306.4869

Epoch 00875: val_loss did not improve from 233.67676
Epoch 876/1000
5706/5706 [==============================] - 1s 107us/step - loss: 145.0360 - mean_absolute_error: 6.4957 - mean_squared_error: 145.0360 - val_loss: 253.7720 - val_mean_absolute_error: 7.3416 - val_mean_squared_error: 253.7720

Epoch 00876: val_loss did not improve from 233.67676
Epoch 877/1000
5706/5706 [==============================] - 1s 105us/step - loss: 145.4177 - mean_absolute_error: 6.3582 - mean_squared_error: 145.4177 - val_loss: 243.7857 - val_mean_absolute_error: 7.1243 - val_mean_squared_error: 243.7857

Epoch 00877: val_loss did not improve from 233.67676
Epoch 878/1000
5706/5706 [==============================] - 1s 109us/step - loss: 146.3983 - mean_absolute_error: 6.4014 - mean


Epoch 00902: val_loss did not improve from 233.67676
Epoch 903/1000
5706/5706 [==============================] - 1s 106us/step - loss: 135.0562 - mean_absolute_error: 6.2478 - mean_squared_error: 135.0562 - val_loss: 329.9244 - val_mean_absolute_error: 8.4770 - val_mean_squared_error: 329.9244

Epoch 00903: val_loss did not improve from 233.67676
Epoch 904/1000
5706/5706 [==============================] - 1s 108us/step - loss: 139.8778 - mean_absolute_error: 6.2355 - mean_squared_error: 139.8778 - val_loss: 247.8368 - val_mean_absolute_error: 7.0241 - val_mean_squared_error: 247.8368

Epoch 00904: val_loss did not improve from 233.67676
Epoch 905/1000
5706/5706 [==============================] - 1s 111us/step - loss: 136.3200 - mean_absolute_error: 6.1867 - mean_squared_error: 136.3200 - val_loss: 269.9222 - val_mean_absolute_error: 7.6251 - val_mean_squared_error: 269.9222

Epoch 00905: val_loss did not improve from 233.67676
Epoch 906/1000
5706/5706 [==============================] 


Epoch 00930: val_loss did not improve from 233.67676
Epoch 931/1000
5706/5706 [==============================] - 1s 107us/step - loss: 157.5931 - mean_absolute_error: 6.5770 - mean_squared_error: 157.5931 - val_loss: 299.5200 - val_mean_absolute_error: 7.7966 - val_mean_squared_error: 299.5200

Epoch 00931: val_loss did not improve from 233.67676
Epoch 932/1000
5706/5706 [==============================] - 1s 105us/step - loss: 129.2971 - mean_absolute_error: 6.0557 - mean_squared_error: 129.2971 - val_loss: 248.7333 - val_mean_absolute_error: 6.8782 - val_mean_squared_error: 248.7333

Epoch 00932: val_loss did not improve from 233.67676
Epoch 933/1000
5706/5706 [==============================] - 1s 109us/step - loss: 131.1202 - mean_absolute_error: 6.0830 - mean_squared_error: 131.1202 - val_loss: 255.2131 - val_mean_absolute_error: 7.0289 - val_mean_squared_error: 255.2131

Epoch 00933: val_loss did not improve from 233.67676
Epoch 934/1000
5706/5706 [==============================] 

5706/5706 [==============================] - 1s 106us/step - loss: 144.9246 - mean_absolute_error: 6.1427 - mean_squared_error: 144.9246 - val_loss: 257.5541 - val_mean_absolute_error: 6.8933 - val_mean_squared_error: 257.5541

Epoch 00958: val_loss did not improve from 230.78113
Epoch 959/1000
5706/5706 [==============================] - 1s 109us/step - loss: 125.2813 - mean_absolute_error: 5.9419 - mean_squared_error: 125.2813 - val_loss: 251.3355 - val_mean_absolute_error: 7.1444 - val_mean_squared_error: 251.3355

Epoch 00959: val_loss did not improve from 230.78113
Epoch 960/1000
5706/5706 [==============================] - 1s 106us/step - loss: 727.6255 - mean_absolute_error: 11.8446 - mean_squared_error: 727.6255 - val_loss: 736.9860 - val_mean_absolute_error: 11.9437 - val_mean_squared_error: 736.9860

Epoch 00960: val_loss did not improve from 230.78113
Epoch 961/1000
5706/5706 [==============================] - 1s 109us/step - loss: 486.3357 - mean_absolute_error: 10.0544 - m


Epoch 00985: val_loss did not improve from 230.78113
Epoch 986/1000
5706/5706 [==============================] - 1s 106us/step - loss: 191.3256 - mean_absolute_error: 7.1397 - mean_squared_error: 191.3256 - val_loss: 347.0500 - val_mean_absolute_error: 8.1491 - val_mean_squared_error: 347.0500

Epoch 00986: val_loss did not improve from 230.78113
Epoch 987/1000
5706/5706 [==============================] - 1s 108us/step - loss: 171.5694 - mean_absolute_error: 6.9447 - mean_squared_error: 171.5694 - val_loss: 354.0985 - val_mean_absolute_error: 8.3450 - val_mean_squared_error: 354.0985

Epoch 00987: val_loss did not improve from 230.78113
Epoch 988/1000
5706/5706 [==============================] - 1s 107us/step - loss: 186.9951 - mean_absolute_error: 7.1400 - mean_squared_error: 186.9951 - val_loss: 315.7263 - val_mean_absolute_error: 7.8668 - val_mean_squared_error: 315.7263

Epoch 00988: val_loss did not improve from 230.78113
Epoch 989/1000
5706/5706 [==============================] 

In [235]:
wights_file = 'Weights-957--230.78113.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [238]:
neural_pred = list(NN_model.predict(X_test)[:,0])

In [239]:
neural_forecast = ({'Actual': Y_test, 'Predicted': neural_pred})

In [240]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, neural_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, neural_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, neural_pred)))

Mean Absolute Error: 6.685244681542334
Mean Squared Error: 159.3846550472619
Root Mean Squared Error: 12.624763564014254


In [225]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_breast_cancer


lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,Y_train)
train_score001=lasso001.score(X_train,Y_train)
test_score001=lasso001.score(X_test,Y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)
print ("training score for alpha=0.01:", train_score001 )
print ("test score for alpha =0.01: ", test_score001)
print ("number of features used: for alpha =0.01:", coeff_used001)

lasso_pred = lasso001.predict(X_test)


training score for alpha=0.01: 0.8610149222279182
test score for alpha =0.01:  0.8666229911149762
number of features used: for alpha =0.01: 33


In [226]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, lasso_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, lasso_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, lasso_pred)))

Mean Absolute Error: 16.68981874885545
Mean Squared Error: 925.5272718302721
Root Mean Squared Error: 30.42247971205293
